In [15]:
#Connect to GIS
from arcgis.gis import GIS
gis = GIS("home")

In [4]:
#Packages
import requests
import pandas as pd
import zipfile
import os
import io
from arcgis import features, GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS
from arcgis.features.analysis import join_features
from arcgis.features import FeatureLayerCollection
from arcgis.geometry import SpatialReference, Geometry
import numpy as np

In [7]:
origin_destination= r'C:\Mac\Home\Documents\ArcGIS\Projects\Lab3_Notebooks\Lab3_Notebooks.gdb\points_XYTableToPoint'

In [8]:
directory=r'C:\Mac\Home\Documents\ArcGIS\Projects\Lab3_Notebooks\Lab3_Notebooks.gdb'

In [9]:
dem= r'C:\Mac\Home\Documents\ArcGIS\Projects\Lab3_Notebooks\Lab3_Notebooks.gdb\digital_elevation_model_Clip'
lc= r'C:\Mac\Home\Documents\ArcGIS\Projects\Lab3_Notebooks\Lab3_Notebooks.gdb\NLCD_2019_Land_Cover_Clip'
bndry = arcpy.ddd.RasterDomain(dem, "aoi", "POLYGON")

# Clip Landcover to AOI
out_lc =r'C:\Mac\Home\Documents\ArcGIS\Projects\Lab3_Notebooks\Lab3_Notebooks.gdb\landcover_aoi'

lc_aoi = arcpy.management.Clip(lc, "564958.63 4875662.12 577613.63 4889678.12", out_lc, bndry, "255", "ClippingGeometry", "MAINTAIN_EXTENT")
     

In [10]:
# Reclassify Landcover
lc_reclassify = arcpy.sa.Reclassify(lc_aoi, "Value", "1 100 5;101 2;102 2;103 2;104 3;105 2;106 3;107 3;108 4;109 1;110 1", "DATA")
     

In [11]:
# Reclassify Slope

dem_reclassify = arcpy.sa.Reclassify(dem, "VALUE", "0 2.084853 5;2.084853 3.822231 4;3.822231 6.602035 3;6.602035 18.416204 2;18.416204 88.606262 1", "DATA") 
     

In [12]:
weights = np.arange(0.1, 1.0, 0.1)
for weight in weights:
    slope_weight = round(weight, 1)
    landcover_weight = round(1 - weight, 1)
    
    cost_surface = ((arcpy.Raster(dem_reclassify) * slope_weight) + 
                    (arcpy.Raster(lc_reclassify) * landcover_weight)) * -1 + 6

    cost_path_name = fr"costPath_{str(slope_weight)[2:3]}_slope_{str(landcover_weight)[2:3]}_landcover"
    cost_path = arcpy.sa.OptimalRegionConnections(origin_destination, cost_path_name, in_cost_raster=cost_surface)
